# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [2]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [3]:
year: int = 2021
topic: str = "FARMACIA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

In [4]:
add_list: list = [x for x in range(17, 78, 1)] + [x for x in range(122, 150, 1)] + [x for x in range(183, 188, 1)] + [x for x in range(192, 201, 1)] + [202, 203, 205, 208]

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [5]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,- 1
1,-
2,1. ¿Qué ventaja tiene el empleo de aciloximeti...
3,teres en lugar de ésteres simples en el diseño de
4,profármacos de ácidos carboxílicos?:
5,1. Disminuye la velocidad de hidrólisis.
6,2. Aumen
7,ta la estabilidad del profármaco.
8,3. El grupo éster externo es
9,más accesible al


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [6]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,3,43,1,85,2.0,127,3,169,3.0
1,2,1,44,1,86,2.0,128,2,170,1.0
2,3,3,45,1,87,2.0,129,1,171,2.0
3,4,4,46,1,88,3.0,130,4,172,1.0
4,5,3,47,1,89,1.0,131,2,173,2.0
5,6,1,48,3,90,2.0,132,3,174,4.0
6,7,3,49,2,91,3.0,133,4,175,3.0
7,8,2,50,2,92,3.0,134,2,176,1.0
8,9,4,51,3,93,1.0,135,3,177,3.0
9,10,4,52,2,94,4.0,136,3,178,2.0


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [7]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains(topic)]

footer_list: list = [f"- {npag} -" for npag in range(0,30,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [8]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [9]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [13]:
raw_exam_df = raw_exam_df[2:]
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [14]:
raw_exam_df.head(10)

0    1. ¿Qué ventaja tiene el empleo de aciloximeti...
1    teres en lugar de ésteres simples en el diseño de
2                 profármacos de ácidos carboxílicos?:
3             1. Disminuye la velocidad de hidrólisis.
4                                             2. Aumen
5                    ta la estabilidad del profármaco.
6                         3. El grupo éster externo es
7                                     más accesible al
8                      centro activo de las esterasas.
9                                     4. Facilita la b
Name: text, dtype: object

In [15]:
raw_exam_df.tail(10)

2067                               1. Carga de los iones.
2068                                2. Masa de los iones.
2069                 3. Relación masa/carga de los iones.
2070                 4. Relación carga/masa de los iones.
2071    210. ¿Qué antídoto se indica en una sobredosis...
2072           anticoagulantes del tipo de la warfarina?:
2073                                     1. Idarucizumab.
2074                                      2. Flumazenilo.
2075                                        3. Andexanet.
2076                                       4. Vitamina K.
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [16]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [17]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. ¿Qué ventaja tiene el empleo de aciloximeti...
1             1. Disminuye la velocidad de hidrólisis.
2           2. Aumen ta la estabilidad del profármaco.
3    3. El grupo éster externo es más accesible al ...
4    4. Facilita la b ioactivación selectiva en el ...
5    2. ¿Qué inconveniente presenta el empleo de ba...
6                          1. Son inestables in vitro.
7             2. Aumen tan excesivamente la lipofilia.
8                  3. So n demasiado estables in vivo.
9    4. Fo rman con facilidad redes cristalinas muy...
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [18]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 5
Number of rows not ending with '.', ':' or '?' = 6
[300, 383, 436, 598, 890, 1019]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [19]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [20]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [22]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    # raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    rows_fix: list[int] = [300, 383, 436, 598, 1019]
    exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

Id to fix 300:
298                                      3. Mieloblasto.
299                                     4. Promielocito.
300    61. ¿Cuál de las siguientes alteraciones morfo...
301                                         100.000/L?:
302                                      1. Esferocitos.
Name: text, dtype: object
Id to fix 383:
381    77. ¿Cuál de las siguientes afirmaciones sobre...
382           1. Produce hiperplasia suprarrenocortical.
383       2. Se reducen las concentraciones urinarias de
384                                    17-cetosteroides.
385                       3. Cursa con hipocortisolismo.
Name: text, dtype: object
Id to fix 436:
434    2. Puede causar hipermagnesemia en tratamiento...
435     3. Puede disminuir la absorción de vitamina B12.
436      4. La máxima inhibición ácida se consigue a los
437                             3-4 días de tratamiento.
438    88. En pacientes en tratamiento para el contro...
Name: text, dtype: object
Id to fix 598:
596    

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [23]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 1050
1050


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [24]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [25]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [26]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [27]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [28]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,1. ¿Qué ventaja tiene el empleo de aciloximeti...,1. Disminuye la velocidad de hidrólisis.,2. Aumen ta la estabilidad del profármaco.,3. El grupo éster externo es más accesible al ...,4. Facilita la b ioactivación selectiva en el ...
1,2. ¿Qué inconveniente presenta el empleo de ba...,1. Son inestables in vitro.,2. Aumen tan excesivamente la lipofilia.,3. So n demasiado estables in vivo.,4. Fo rman con facilidad redes cristalinas muy...
2,3. ¿Cuál de los siguientes sustituyentes prese...,1. Bifenilo.,2. Hidrox imetilo.,3. Tetrazol.,4. n -Butilo.
3,4. ¿Qué papel desempeñan los fragmentos de bet...,1. Establecen enl aces covalentes con el ADN.,2. Establecen enl aces de hidrógeno con la top...,3. Fijan la conformación activa a través de en...,4. Forman quelatos con el Fe 3+ favoreciendo l...
4,5. ¿Qué consecuencia tiene el cambio de posici...,1. Se fa vorece la degradación por la catecol-...,2. Se pierde actividad al disminuir la analogí...,3. Aumen ta la estabilidad metabólica.,4. Se favo rece la degr adación por monoami no...
5,6. El catión metildiazonio es un metabolito ac...,1. Procarbazina.,2. C iclofosfamida.,3. Mecloretamina .,4. Busulfán.
6,7. ¿Cuál de los siguientes fármacos se conside...,1. Lo pinavir.,2. Captopril.,3. Seleg ilina.,4. R ivastigmina.
7,8. ¿Cuál de las siguientes reacciones metabóli...,1. Gl ucuronidación.,2. Acetilación.,3. C onjugación con sulfato.,4. Conjugación con glutatión.
8,9. Un método general para obtener derivados de...,1. Fen ilhidrazina y un compuesto carbon ílico...,"2. Un a 1,3-diamina y carbonato de etilo.","3. Acetilacetato de etilo, amoniaco y alde híd...",4. Un éster malón ico convenientemente sustitu...
9,10. Las sulfonilureas ejercen su efecto hipogl...,1. Canales de calcio dependientes de potencial.,2. Canales de sodio dependientes de potencial.,3. B ombas iónicas dependientes de ATP.,4. Canales de potasio dependientes de ATP.


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [29]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [30]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [31]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [32]:
answers_df_clean.head(10)

,V,RC
0,1,3.0
1,2,1.0
2,3,3.0
3,4,4.0
4,5,3.0
5,6,1.0
6,7,3.0
7,8,2.0
8,9,4.0
9,10,4.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [33]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,1. ¿Qué ventaja tiene el empleo de aciloximeti...,1. Disminuye la velocidad de hidrólisis.,2. Aumen ta la estabilidad del profármaco.,3. El grupo éster externo es más accesible al ...,4. Facilita la b ioactivación selectiva en el ...,3.0
1,2. ¿Qué inconveniente presenta el empleo de ba...,1. Son inestables in vitro.,2. Aumen tan excesivamente la lipofilia.,3. So n demasiado estables in vivo.,4. Fo rman con facilidad redes cristalinas muy...,1.0
2,3. ¿Cuál de los siguientes sustituyentes prese...,1. Bifenilo.,2. Hidrox imetilo.,3. Tetrazol.,4. n -Butilo.,3.0
3,4. ¿Qué papel desempeñan los fragmentos de bet...,1. Establecen enl aces covalentes con el ADN.,2. Establecen enl aces de hidrógeno con la top...,3. Fijan la conformación activa a través de en...,4. Forman quelatos con el Fe 3+ favoreciendo l...,4.0
4,5. ¿Qué consecuencia tiene el cambio de posici...,1. Se fa vorece la degradación por la catecol-...,2. Se pierde actividad al disminuir la analogí...,3. Aumen ta la estabilidad metabólica.,4. Se favo rece la degr adación por monoami no...,3.0
5,6. El catión metildiazonio es un metabolito ac...,1. Procarbazina.,2. C iclofosfamida.,3. Mecloretamina .,4. Busulfán.,1.0
6,7. ¿Cuál de los siguientes fármacos se conside...,1. Lo pinavir.,2. Captopril.,3. Seleg ilina.,4. R ivastigmina.,3.0
7,8. ¿Cuál de las siguientes reacciones metabóli...,1. Gl ucuronidación.,2. Acetilación.,3. C onjugación con sulfato.,4. Conjugación con glutatión.,2.0
8,9. Un método general para obtener derivados de...,1. Fen ilhidrazina y un compuesto carbon ílico...,"2. Un a 1,3-diamina y carbonato de etilo.","3. Acetilacetato de etilo, amoniaco y alde híd...",4. Un éster malón ico convenientemente sustitu...,4.0
9,10. Las sulfonilureas ejercen su efecto hipogl...,1. Canales de calcio dependientes de potencial.,2. Canales de sodio dependientes de potencial.,3. B ombas iónicas dependientes de ATP.,4. Canales de potasio dependientes de ATP.,4.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [34]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [35]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [36]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [37]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. ¿Qué ventaja tiene el empleo de aciloximeti...,1. Disminuye la velocidad de hidrólisis.,2. Aumen ta la estabilidad del profármaco.,3. El grupo éster externo es más accesible al ...,4. Facilita la b ioactivación selectiva en el ...,3,2021
1,2. ¿Qué inconveniente presenta el empleo de ba...,1. Son inestables in vitro.,2. Aumen tan excesivamente la lipofilia.,3. So n demasiado estables in vivo.,4. Fo rman con facilidad redes cristalinas muy...,1,2021
2,3. ¿Cuál de los siguientes sustituyentes prese...,1. Bifenilo.,2. Hidrox imetilo.,3. Tetrazol.,4. n -Butilo.,3,2021
3,4. ¿Qué papel desempeñan los fragmentos de bet...,1. Establecen enl aces covalentes con el ADN.,2. Establecen enl aces de hidrógeno con la top...,3. Fijan la conformación activa a través de en...,4. Forman quelatos con el Fe 3+ favoreciendo l...,4,2021
4,5. ¿Qué consecuencia tiene el cambio de posici...,1. Se fa vorece la degradación por la catecol-...,2. Se pierde actividad al disminuir la analogí...,3. Aumen ta la estabilidad metabólica.,4. Se favo rece la degr adación por monoami no...,3,2021
5,6. El catión metildiazonio es un metabolito ac...,1. Procarbazina.,2. C iclofosfamida.,3. Mecloretamina .,4. Busulfán.,1,2021
6,7. ¿Cuál de los siguientes fármacos se conside...,1. Lo pinavir.,2. Captopril.,3. Seleg ilina.,4. R ivastigmina.,3,2021
7,8. ¿Cuál de las siguientes reacciones metabóli...,1. Gl ucuronidación.,2. Acetilación.,3. C onjugación con sulfato.,4. Conjugación con glutatión.,2,2021
8,9. Un método general para obtener derivados de...,1. Fen ilhidrazina y un compuesto carbon ílico...,"2. Un a 1,3-diamina y carbonato de etilo.","3. Acetilacetato de etilo, amoniaco y alde híd...",4. Un éster malón ico convenientemente sustitu...,4,2021
9,10. Las sulfonilureas ejercen su efecto hipogl...,1. Canales de calcio dependientes de potencial.,2. Canales de sodio dependientes de potencial.,3. B ombas iónicas dependientes de ATP.,4. Canales de potasio dependientes de ATP.,4,2021


### Filtering by relevant questions

**1. Filtering for id**

A list of the number of question is used to filter the DataFrame, but first doing a -1 so it matches with the id. 

Finally the indexes are reseted

In [38]:
id_filter = [x-1 for x in add_list]
processed_exam_df_filtered = processed_exam_df.iloc[id_filter]
processed_exam_df_filtered = processed_exam_df_filtered.reset_index(drop= True)
print(f"Expected number of rows = {len(id_filter)}\nNumber of rows in filtered df = {processed_exam_df_filtered.shape[0]}")

Expected number of rows = 107
Number of rows in filtered df = 107


**2. Showing the filtered DataFrame**

Using head and tail

In [39]:
processed_exam_df_filtered.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,17. La anfotericina B es un antifúngico que pe...,1. Polienos.,2. Triazoles.,3. Candinas.,4. Sulfamidas.,1,2021
1,18. Respecto a los hongos Mucorales:,1. Son sensibles a voriconazol.,2. Causan infecciones diagnosticables mediante...,"3. Causan infección sinusal, rino-órbitocerebr...",4. Tienen afinidad por tejidos queratinizados.,3,2021
2,19. ¿Qué mecanismo es responsable de la resist...,1. Presencia de mutaciones cromosómicas en los...,2. Modificación enzimática de la quinolona med...,3. Adquisición de una β-lactamasa que hidroliz...,"4. Metilación del ARNr 23S, impidiendo la unió...",1,2021
3,20. ¿Cuál de los siguientes enunciados respect...,1. Se identifica en la mayoría de cepas de S. ...,2. Es causante de cuadros cutáneos graves.,3. Es una citotoxina.,4. Se relaciona con intoxicaciones alimentarias.,4,2021
4,21. ¿Cuál es la respuesta correcta respecto a ...,1. Produce infecciones del aparato genitourina...,2. Es sensible a fosfomicina.,3. Es sensible a novobiocina.,4. Es un estafilococo coagulasa-positivo.,1,2021
5,22. ¿Cuál de las siguientes bacterias es un ba...,1. Pasteurella multocida.,2. Francisella tularensis.,3. Neisseria gonorrhoeae.,4. Enterobacter cloacae.,1,2021
6,23. ¿Cuál es la respuesta INCORRECTA respecto ...,1. Son asintomáticas con relativa frecuencia.,2. La infección sintomática se presenta en una...,3. El tratamiento de elección son los macrólid...,4. La fiebre de Pontiac se asocia con enfermed...,4,2021
7,24. ¿Cuál de las siguientes bacterias NO perte...,1. Achromobacter xylosoxidans.,2. Cardiobacterium hominis.,3. Kingella kingae.,4. Eikenella corrodens.,1,2021
8,25. ¿Cuál de las siguientes características fe...,"1. β-Hemólisis en agar sa ngre, sensibilidad a...","2. β-Hemólisis en agar sangre, resistencia a l...","3. α-Hemólisis en agar sangre, sensibilidad a ...","4. α-Hemólisis en agar sangre, resistencia a l...",3,2021
9,26. Avibactam es un inhibidor de β-lactamasas ...,1. β-Lactamasa de espectro extendido (BLEE).,2. β-Lactamasa de clase C (ampC).,3. Carbapenemasa de tipo metalobetalactamasa.,4. Carbapenemasa de clase D (OXA-48).,3,2021


In [40]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
97,195. ¿En cuál de las siguientes situaciones NO...,1. Bacteria gram positiva reconocida por IgM.,2. Bacteria gram negativa reconocida por IgA.,3. Bacteria gram positiva opsonizada con IgG.,4. Bacteria gram negativa opsonizada por Prote...,2,2021
98,196. ¿Qué enunciado es una característica de l...,1. Son de proliferación constante sin antígeno...,"2. Son quiescentes, de reactividad aumentada a...",3. Son quiescentes y de reactividad aumentada ...,4. Son inmortales y activan a las células T ví...,3,2021
99,197. ¿Qué células sanguíneas circulantes se di...,1. Neutrófilos.,2. Macrófagos inmaduros.,3. Monocitos.,4. Células mieloides progenitoras circulantes.,3,2021
100,198. ¿Cuál es la función principal del timo?:,1. Es el órgano donde los linfocitos T senesce...,2. Es el órgano donde residen los linfocitos d...,3. Es el órgano donde se induce la activación ...,4. Es el órgano donde se produce la maduración...,4,2021
101,"199. Las células dendrític as, durante una inf...","1. Fagocitar microbios y erradicarlos, secreta...",2. Migrar desde el sitio de infección hasta el...,3. Llevar desde el sitio de infección hasta el...,4. Presentar antígenos a los linfocitos T en e...,3,2021
102,200. ¿Cuál es la función de los linfocitos CD4...,"1. Producir IL-1, IL-7 e inducir una respuesta...","2. Producir IL-1, IL-7 e inducir una respuesta...",3. Producir IL-17 e inducir una respuesta infl...,4. Producir IL-17 e inducir una respuesta anti...,3,2021
103,202. ¿Cuál de las siguie ntes afirmaciones sob...,1. Es un bacilo grampositivo.,2. Forma colonias catalasa y CAMP positivas.,3. No crece en hemocultivos.,4. Presenta resistencia intrínseca a la ceftri...,3,2021
104,203. La vía aferente del re flejo fotomoto r c...,1. Núcleo geniculado.,2. Núcleo de Edinger-Westphal.,3. Núcleo ciliespinal de Budge.,4. Núcleo de Deiters.,2,2021
105,205. ¿Cuál de las siguientes afirmaciones acer...,1. Es un antibiótico del grupo de los lipopépt...,2. Se utiliza por vía oral en el tratamiento d...,"3. Produce como efectos adversos exantemas, ot...",4. Su mecanismo de acción es inhibir la síntes...,1,2021
106,208. ¿Cuál de las siguientes proteínas plasmát...,1. Prealbúmina.,2. Haptoglobina.,3. Ferroxidasa.,4. Proteína transportadora de retinol.,1,2021


# Data Loading

## Saving in csv file

The `processed_exam_df_filtered` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [41]:
processed_exam_df_filtered.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [42]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [43]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT id FROM questions WHERE question = ?),
    ?,
    ?,
    ?)
"""

**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

In [44]:

with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df_filtered.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], n+1, option, n+1 == question[6]))
            bir_warehouse.commit()